## 1) Składanie i "aplikacja" funkcji

In [1]:
(<$<) :: (a -> b) -> a -> b
(<$<) = ($)

(>$>) :: a -> (a -> b) -> b
x >$> f = f x
infixl 0 >$>

In [4]:
(<.<) :: (b -> c) -> (a -> b) -> (a -> c)
(<.<) = (.)

(>.>) :: (a -> b) -> (b -> c) -> (a -> c)
f >.> g = g . f
infixl 9 >.>

In [15]:
safeTail :: [a] -> Maybe [a]
safeTail [] = Nothing
safeTail (x:xs) = Just xs

extractMaybe :: Maybe a -> a
extractMaybe Nothing = error "Nothing inside!"
extractMaybe (Just x) = x

insertMaybe :: a -> Maybe a
insertMaybe = Just

(>^$>) :: Maybe a -> (a -> Maybe b) -> Maybe b
Nothing >^$> _ = Nothing
(Just x) >^$> f = f x
infixl 1 >^$>

In [18]:
f1 :: (Ord a, Num a) => a -> Maybe a
f1 x = if x > 0 then Just (x + 1) else Nothing

f2 :: (Eq a, Num a) => a -> Maybe a
f2 x = if x /= 0 then Just (10 * x) else Nothing

(>.>>) :: (a -> Maybe b) -> (b -> Maybe c) -> (a -> Maybe c)
f >.>> g = \x -> g (extractMaybe (f x))

Line 8: Avoid lambda
Found:
\ x -> g (extractMaybe (f x))
Why not:
g . extractMaybe . f

### 1.10.1

In [20]:
(>.>>) :: (a -> Maybe b) -> (b -> Maybe c) -> (a -> Maybe c)
f >.>> g = \x -> (f x) >^$> g

Line 2: Redundant bracket
Found:
(f x) >^$> g
Why not:
f x >^$> g

### 1.10.3
Nie ma to większego sensu. Gdyby nagle pure funkcje mogły sobie wyciągać wartości z IO, to nie były by one już pure.

## 2) Przykłady monad: `Maybe`

In [25]:
import Control.Monad
:t (>=>)

(>=>) :: forall (m :: * -> *) a b c. Monad m => (a -> m b) -> (b -> m c) -> a -> m c

In [28]:
:t (>>=)

(>>=) :: forall (m :: * -> *) a b. Monad m => m a -> (a -> m b) -> m b

### 2.11.1

In [36]:
(>=>) :: (a -> Maybe b) -> (b -> Maybe c) -> (a -> Maybe c)
f >=> g = \x -> f x >>= g

Line 2: Use >=>
Found:
\ x -> f x >>= g
Why not:
f Control.Monad.>=> g

Tak, bo [tak jest najwyraźniej zdefiniowany](https://wiki.haskell.org/Monad_laws)

### 2.11.3

In [46]:
join :: Maybe (Maybe a) -> Maybe a
join Nothing = Nothing
join (Just Nothing) = Nothing
join (Just (Just x)) = Just x

## 4) Przykłady monad: `[]`

### 4.6.2

In [47]:
join :: [[a]] -> [a]
join [] = []
join (x:xs) = x ++ join xs

Line 2: Use foldr
Found:
join [] = []
join (x : xs) = x ++ join xs
Why not:
join xs = foldr (++) [] xs